<style>
table, th, td {
    border: 1px solid black;
}
</style>

<table style="width:100%" >

  <tr >
    <th align="center">Tensorflow</th>
    <th align="center">python</th>
    <th align="center">google</th>
    <th align="center">udacity</th>
  </tr>

  <tr>
    <td>
      <img src="https://upload.wikimedia.org/wikipedia/commons/a/a4/TensorFlowLogo.png" height="200" width="200" />
    </td>
    <td>
      <img src="http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png" height="200" width="200" />
    </td>
    <td>
      <img src="https://www.blog.google/static/blog/images/google-200x200.7714256da16f.png" height="200" width="200" />
    </td>
    <td>
      <img src="https://d125fmws0bore1.cloudfront.net/assets/udacity_share-317a7f82552763598a5c91e1550b7cd83663ce02d6d475d703e25a873cd3b574.png" height="200" width="200" />
    </td>
  </tr>
</table>

# Deep Learning - my Notes.
## Arturo Alatriste Trujillo

In [1]:
%pwd

'C:\\Users\\a.alatriste.trujillo\\dl_udacity'

# Deep Learning
## Assignment 3

Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.
The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy      as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels  = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels  = save['valid_labels']
  test_dataset  = save['test_dataset']
  test_labels   = save['test_labels']
  del save  # hint to help gc free up memory
  print( 'Training set'  , train_dataset.shape, train_labels.shape )
  print( 'Validation set', valid_dataset.shape, valid_labels.shape )
  print( 'Test set'      , test_dataset.shape , test_labels.shape  )

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
data as a flat matrix,
labels as float 1-hot encodings.

In [5]:
# images of 28 x 28 pixels
image_size = 28

# we want to classify bewtween 10 letters: a, b, c, d, e, f, g, h, i, j.
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat( train_dataset, train_labels )
valid_dataset, valid_labels = reformat( valid_dataset, valid_labels )
test_dataset , test_labels  = reformat( test_dataset , test_labels  )

print( 'Training set'  , train_dataset.shape, train_labels.shape )
print( 'Validation set', valid_dataset.shape, valid_labels.shape )
print( 'Test set'      , test_dataset.shape, test_labels.shape   )

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum( np.argmax(predictions, 1) == np.argmax(labels, 1) )
          / predictions.shape[0])

---

### Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

<img src="img/dnn12.png" height="600" width="600" />

#### logistic model - graph

In [12]:
# This is to expedite the process 
train_subset = 10000
# This is a good beta value to start with
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # They're all constants.
    tf_train_dataset = tf.constant( train_dataset[:train_subset, :] )
    tf_train_labels  = tf.constant( train_labels[:train_subset]     )
    tf_valid_dataset = tf.constant( valid_dataset                   )
    tf_test_dataset  = tf.constant( test_dataset                    )


    # Variables    
    # They are variables we want to update and optimize.
    weights          = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases           = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits           = tf.matmul(tf_train_dataset, weights) + biases 
    # Original loss function
    loss             = tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits( labels=tf_train_labels, logits=logits) )
    
    # Loss function using L2 Regularization
    regularizer      = tf.nn.l2_loss(weights)
    loss             = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction  = tf.nn.softmax( tf.matmul(tf_test_dataset, weights) + biases)

#### logistic model - run

In [14]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step {}: {}'.format(step, l))
            print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                         train_labels[:train_subset, :])))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            
            # You don't have to do .eval above because we already ran the session for the
            # train_prediction
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 46.304771423339844
Training accuracy: 12.1
Validation accuracy: 14.6
Loss at step 100: 11.719772338867188
Training accuracy: 73.5
Validation accuracy: 71.9
Loss at step 200: 4.437258243560791
Training accuracy: 78.9
Validation accuracy: 76.5
Loss at step 300: 1.9617443084716797
Training accuracy: 82.4
Validation accuracy: 80.0
Loss at step 400: 1.1219197511672974
Training accuracy: 83.8
Validation accuracy: 81.5
Loss at step 500: 0.8335093259811401
Training accuracy: 84.7
Validation accuracy: 81.9
Loss at step 600: 0.7332196831703186
Training accuracy: 84.8
Validation accuracy: 82.1
Loss at step 700: 0.6980148553848267
Training accuracy: 84.9
Validation accuracy: 82.1
Loss at step 800: 0.6855627298355103
Training accuracy: 84.9
Validation accuracy: 82.1
Test accuracy: 88.8


#### Neural Network with L2 Regularization

* 1 Hidden Layer using RELUs

In [15]:
num_nodes= 1024
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)

    # Variables.
    weights_1        = tf.Variable( tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1         = tf.Variable( tf.zeros([num_nodes]))
    weights_2        = tf.Variable( tf.truncated_normal([num_nodes, num_labels]))
    biases_2         = tf.Variable( tf.zeros([num_labels]))

    # Training computation.
    logits_1         = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer       = tf.nn.relu(logits_1)
    logits_2         = tf.matmul(relu_layer, weights_2) + biases_2
    
    # Normal loss function
    loss             = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits( labels=tf_train_labels, logits=logits_2 ))
    # Loss function with L2 Regularization with beta=0.01
    regularizers     = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss             = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer        = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training
    train_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for validation 
    logits_1         = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer       = tf.nn.relu(logits_1)
    logits_2         = tf.matmul(relu_layer, weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits_2)
    
    # Predictions for test
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    
    test_prediction =  tf.nn.softmax(logits_2)

In [16]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 3523.0703125
Minibatch accuracy: 8.6
Validation accuracy: 32.0
Minibatch loss at step 500: 21.19504165649414
Minibatch accuracy: 86.7
Validation accuracy: 84.5
Minibatch loss at step 1000: 0.9585634469985962
Minibatch accuracy: 81.2
Validation accuracy: 83.0
Minibatch loss at step 1500: 0.5872321128845215
Minibatch accuracy: 87.5
Validation accuracy: 83.5
Minibatch loss at step 2000: 0.6100964546203613
Minibatch accuracy: 89.8
Validation accuracy: 83.4
Minibatch loss at step 2500: 0.7124186754226685
Minibatch accuracy: 84.4
Validation accuracy: 83.3
Minibatch loss at step 3000: 0.7718384861946106
Minibatch accuracy: 82.0
Validation accuracy: 83.4
Test accuracy: 90.4


### Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

### Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.
What happens to our extreme overfitting case?

### Problem 4

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.
One avenue you can explore is to add multiple layers.
Another one is to use learning rate decay:

``` 
global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
```


#### references

other students
* http://www.ritchieng.com/machine-learning/deep-learning/tensorflow/regularization/
    